In [ ]:
import ipywidgets as widgets

import requests

x_widget = widgets.IntText(value=1)
y_widget = widgets.IntText(value=2)
solution_widget = widgets.IntText(description="Solution:", 
                                  value=x_widget.value * y_widget.value, 
                                  disabled=True)
output = widgets.Output()
box = widgets.VBox(children=[x_widget, y_widget, solution_widget, output])
box

### Handlers
The first handler will use `execute_notebook` and pass in `overrides` to save the cost of recomputing expensive calculations.  

The second handler will hit the `/restified/` endpoint of the same model, which doesn't have any way of recomputing expensive calculations.

The point of this demo is to highlight that even though a model can be turned into a REST endpoint, if you are using the model notebook for callbacks in a Dashboard style app, it is better to use `execute_notebook`.

In [ ]:
import time
from notebook_restified import execute_notebook

precomputed_data = 'some object' # simulating the lengthy setup call here

def handler1(w):
    start = time.time()
    nb = 'model.ipynb'
    params = {'x' : x_widget.value,
              'y' : y_widget.value}
    overrides = {'initialized_data' : precomputed_data}
    solution = execute_notebook(nb, params, overrides)
    solution_widget.value = solution
    duration = time.time() - start
    with output:
        print("Calculated solution using execute_notebook with overrides, took %.2f seconds" % duration)
        
x_widget.observe(handler1, names='value')

In [ ]:
### TODO: figure out how to auth and generate url programmatically in the next few cells

In [ ]:
import requests

### Auth to set cookies
session = requests.Session()
auth_url = 'http://localhost:8888/?token=1536c2c66891aaba71c5ab5f1d4b3ec1c7b694b43f54aff0'
resp = session.get(auth_url)
resp

In [ ]:
model_url = 'http://localhost:8888/notebooks/demo/rest_vs_execute_notebook/model.ipynb'
restified_url = model_url.replace('/notebooks/', '/restified/')
restified_url

In [ ]:
params = {'x' : 1, 'y' : 3}
resp = session.get(restified_url, params=params)
resp

In [ ]:
resp.text

In [ ]:
def handler2(w):
    start = time.time()
    params = {'x' : x_widget.value,
              'y' : y_widget.value}
    resp = session.get(restified_url, params=params)
    solution = resp.text
    solution_widget.value = int(solution)
    duration = time.time() - start
    with output:
        print("Calculated solution using requests.py without overrides, took %.2f seconds" % duration)
        
y_widget.observe(handler2, names='value')